In [2]:
# ============================================================
# modeling_v13_lgbm_hybrid.py
# - v10 pairs (BayesOpt 필터 반영된 pairs_v10_best.csv) 사용
# - 추가 EDA 필터링 없음 (전처리에서 이미 최적 필터 적용)
# - train_month(B안) 기반 pivot → pair별 시계열 feature
# - 팀원 코드의 leader alignment + rich FE + MAE objective
# - 우리 쪽 seed ensemble 구조 결합
# - sample_submission 미사용: 우리가 찾은 pair만 예측
# - value >= 1000 인 pair만 최종 제출
# ============================================================

import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error


# ============================================================
# 0. PATH 설정
# ============================================================
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

TRAIN_MONTH_PATH = DATA_DIR / "processed" / "train_month.csv"          # B안 결과
PAIRS_PATH       = DATA_DIR / "processed" / "v10_pairs" / "pairs_v10_best.csv"

OUTPUT_DIR       = DATA_DIR / "processed" / "v13_model_output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


# ============================================================
# 1. train_month → pivot 생성
#    (전처리에서 만든 train_month.csv 기반, hs4 포함)
# ============================================================
def load_monthly_data():
    df = pd.read_csv(TRAIN_MONTH_PATH)

    df["year"] = df["year"].astype(int)
    df["month"] = df["month"].astype(int)

    df["ym"] = pd.to_datetime(
        df["year"].astype(str) + "-" + df["month"].astype(str) + "-01"
    )

    # item_id × ym pivot (value 합계)
    pivot = (
        df.pivot_table(
            index="ym",
            columns="item_id",
            values="value",
            aggfunc="sum",
        )
        .sort_index()
        .fillna(0.0)
    )

    return df, pivot


# ============================================================
# 2. pair 단위 feature frame 생성 (팀원 alignment + 우리 구조)
# ============================================================
def _build_pair_frame(
    pivot: pd.DataFrame,
    leader: str,
    follower: str,
    best_lag: int,
    max_corr: float,
) -> pd.DataFrame:
    """
    - follower(b_t)를 기준으로 rich time-series feature 생성
    - leader는 best_lag을 고려한 'aligned' 타이밍으로 정렬
    - target: follower(t+1) 의 log1p(value)
    """

    a_raw = pivot[leader]      # leader at t
    b_t   = pivot[follower]    # follower at t

    df = pd.DataFrame(
        {
            "date": pivot.index,
            "b_t": b_t.values,
            "a_raw": a_raw.values,   # 참고용 (feature에서는 제외 예정)
        }
    )

    # -----------------------------
    # Follower features
    # -----------------------------
    df["b_t_1"] = df["b_t"].shift(1)
    df["b_t_2"] = df["b_t"].shift(2)

    df["b_diff1"] = df["b_t"] - df["b_t_1"]
    df["b_diff2"] = df["b_t"] - df["b_t_2"]

    df["b_pct1"] = df["b_diff1"] / (df["b_t_1"].replace(0, np.nan) + 1e-6)
    df["b_pct2"] = df["b_diff2"] / (df["b_t_2"].replace(0, np.nan) + 1e-6)

    df["b_roll3"] = df["b_t"].rolling(window=3, min_periods=1).mean()
    df["b_std3"] = df["b_t"].rolling(window=3, min_periods=1).std()

    df["b_expanding_mean"] = df["b_t"].expanding(min_periods=1).mean()
    df["b_zscore3"] = (df["b_t"] - df["b_roll3"]) / (df["b_std3"] + 1e-6)

    # -----------------------------
    # Leader alignment (핵심)
    # -----------------------------
    # 원래 best_lag 은 leader(t-lag) vs follower(t) 기준으로 찾음.
    # 우리가 예측하려는 것은 follower(t+1).
    # follower(t+1)에 대응하는 leader 시점은 leader(t+1 - lag).
    # row t에서 사용할 leader 정보는 leader(t+1-lag) → index 기준으로 shift_amount = lag-1
    shift_amount = int(best_lag) - 1
    df["a_t_aligned"] = df["a_raw"].shift(shift_amount)

    df["a_t_aligned_1"] = df["a_t_aligned"].shift(1)
    df["a_t_aligned_2"] = df["a_t_aligned"].shift(2)

    df["a_diff1"] = df["a_t_aligned"] - df["a_t_aligned_1"]
    df["a_pct1"] = df["a_diff1"] / (df["a_t_aligned_1"].replace(0, np.nan) + 1e-6)
    df["a_roll3"] = df["a_t_aligned"].rolling(window=3, min_periods=1).mean()

    # -----------------------------
    # Interaction features
    # -----------------------------
    df["ab_ratio"] = df["a_t_aligned"] / (df["b_t"] + 1e-6)
    df["ab_diff"] = df["a_t_aligned"] - df["b_t"]

    # -----------------------------
    # Seasonality-ish (month, quarter)
    # -----------------------------
    df["month"] = df["date"].dt.month
    df["quarter"] = df["date"].dt.quarter

    # -----------------------------
    # Target: follower(t+1)
    # -----------------------------
    df["target_value"] = df["b_t"].shift(-1)
    df["target_log1p"] = np.log1p(df["target_value"].clip(lower=0))
    df["target_date"] = df["date"] + pd.offsets.MonthBegin(1)

    # Meta
    df["leading_item_id"] = leader
    df["following_item_id"] = follower
    df["best_lag"] = best_lag
    df["max_corr"] = max_corr

    return df


def build_training_data(pivot: pd.DataFrame, pairs_df: pd.DataFrame) -> pd.DataFrame:
    frames = []
    for _, row in pairs_df.iterrows():
        pair_frame = _build_pair_frame(
            pivot=pivot,
            leader=row["leading_item_id"],
            follower=row["following_item_id"],
            best_lag=int(row["best_lag"]),
            max_corr=float(row["max_corr"]),
        )

        # 최소한의 history 확보 (b_t_2, a_t_aligned_2 존재하는 구간만)
        pair_frame = pair_frame.dropna(subset=["b_t_2", "a_t_aligned_2"])

        # 너무 짧은 시계열 pair는 제거
        if len(pair_frame) <= 8:
            continue

        frames.append(pair_frame)

    if not frames:
        raise ValueError("훈련 데이터 없음. (모든 pair frame이 비었음)")

    train_df = pd.concat(frames, ignore_index=True)

    # target 없는 마지막 구간 제거
    train_df = train_df[train_df["target_log1p"].notnull()].reset_index(drop=True)

    return train_df


# ============================================================
# 3. LGBM seed ensemble (MAE objective)
# ============================================================
def train_single_lgbm(X_train, y_train, X_valid, y_valid, seed):

    model = LGBMRegressor(
        objective="regression_l1",   # MAE 기반
        n_estimators=2000,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=seed,
        n_jobs=-1,
    )

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="mae",
    )

    pred_valid = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, pred_valid)
    print(f"[Seed {seed}] Valid MAE (log1p-space) = {mae:.5f}")

    return model, mae


def train_lgbm_seed_ensemble(
    train_df: pd.DataFrame,
    seeds = (42, 2024, 777),
):
    # feature / target 분리
    drop_cols = {
        "date",
        "target_date",
        "target_value",
        "target_log1p",
        "leading_item_id",
        "following_item_id",
        "a_raw",        # 원시 leader 값은 피처에서 제외
    }
    feature_cols = [c for c in train_df.columns if c not in drop_cols]

    # time-based split
    train_cutoff = pd.Timestamp("2024-12-01")
    valid_start  = pd.Timestamp("2025-01-01")
    valid_end    = pd.Timestamp("2025-05-01")

    base_mask = train_df["target_log1p"].notnull()

    train_mask = (train_df["target_date"] <= train_cutoff) & base_mask
    valid_mask = (
        (train_df["target_date"] >= valid_start)
        & (train_df["target_date"] <= valid_end)
        & base_mask
    )

    X_train = train_df.loc[train_mask, feature_cols]
    y_train = train_df.loc[train_mask, "target_log1p"]

    X_valid = train_df.loc[valid_mask, feature_cols]
    y_valid = train_df.loc[valid_mask, "target_log1p"]

    if len(X_train) == 0 or len(X_valid) == 0:
        raise ValueError("train/valid 분할 후 데이터가 없습니다. (기간 설정 확인 필요)")

    print(f"[Split] Train: {X_train.shape}, Valid: {X_valid.shape}")

    models, maes = [], []
    for seed in seeds:
        model, mae = train_single_lgbm(X_train, y_train, X_valid, y_valid, seed)
        models.append(model)
        maes.append(mae)

    print(f"[Ensemble] Avg MAE (log1p-space) = {np.mean(maes):.5f}")
    return models, feature_cols


def predict_ensemble(models, X):
    preds = [m.predict(X) for m in models]
    return np.mean(preds, axis=0)


# ============================================================
# 4. inference: 2025-08 타깃 row feature 생성
# ============================================================
def build_inference_features(
    pivot: pd.DataFrame,
    pairs_df: pd.DataFrame,
    forecast_month: str = "2025-08",
) -> pd.DataFrame:
    """
    - forecast_month: 'YYYY-MM' 문자열
    - target_date == forecast_month-01 인 row를 찾기 위해,
      base_date = target_date - 1 month 를 기준으로 row 하나를 뽑는다.
    """

    target_ts = pd.Timestamp(forecast_month + "-01")
    base_ts = target_ts - pd.offsets.MonthBegin(1)   # 예: 2025-08 → 2025-07-01

    rows = []
    for _, row in pairs_df.iterrows():
        pair_frame = _build_pair_frame(
            pivot=pivot,
            leader=row["leading_item_id"],
            follower=row["following_item_id"],
            best_lag=int(row["best_lag"]),
            max_corr=float(row["max_corr"]),
        )

        # history 확보
        pair_frame = pair_frame.dropna(subset=["b_t_2", "a_t_aligned_2"])

        # base_ts 에 해당하는 row 추출
        target_row = pair_frame[pair_frame["date"] == base_ts]
        if not target_row.empty:
            rows.append(target_row.iloc[0])

    if not rows:
        return pd.DataFrame()

    pred_df = pd.DataFrame(rows).reset_index(drop=True)
    return pred_df


# ============================================================
# 5. 제출 생성 (value >= 1000만 남김)
# ============================================================
def create_submission(
    pairs_df: pd.DataFrame,
    pred_df: pd.DataFrame,
    models,
    feature_cols,
    min_value: int = 1000,
) -> pd.DataFrame:
    if pred_df.empty:
        raise ValueError("pred_df 비어 있음. (inference feature 없음)")

    # feature 매칭용 merge
    sub = pairs_df[["leading_item_id", "following_item_id"]].copy()

    sub = sub.merge(
        pred_df[["leading_item_id", "following_item_id"] + feature_cols],
        on=["leading_item_id", "following_item_id"],
        how="inner",
    )

    if len(sub) == 0:
        raise ValueError("merge 후 예측 가능한 pair가 없습니다.")

    X_test = sub[feature_cols].fillna(0.0)
    y_pred_log = predict_ensemble(models, X_test)
    y_pred = np.expm1(y_pred_log)  # 역 log1p
    y_pred = np.maximum(0, y_pred)

    sub["value"] = y_pred.round().astype(int)

    print(f"[INFO] raw submission rows: {len(sub)}")

    # 🔥 value ≥ min_value 인 pair만 남김
    sub = sub[sub["value"] >= min_value].reset_index(drop=True)
    print(f"[INFO] filtered submission rows (value >= {min_value}): {len(sub)}")

    return sub[["leading_item_id", "following_item_id", "value"]]


# ============================================================
# 6. MAIN
# ============================================================
def main():
    print("=== [1] Load monthly & pivot ===")
    _, pivot = load_monthly_data()

    print("=== [2] Load pairs_v10_best (BayesOpt 필터 반영본) ===")
    pairs_df = pd.read_csv(PAIRS_PATH)
    print(f"[INFO] pairs_v10_best rows: {len(pairs_df)}")

    # 추가 EDA 필터링 없음 (이미 전처리에서 best 필터 적용)

    print("=== [3] Build training df (hybrid FE) ===")
    train_df = build_training_data(pivot, pairs_df)
    print(f"[INFO] training rows: {len(train_df)}")

    print("=== [4] Train LGBM seed ensemble (MAE objective) ===")
    models, feature_cols = train_lgbm_seed_ensemble(train_df)

    print("=== [5] Build inference features for 2025-08 ===")
    pred_df = build_inference_features(pivot, pairs_df, forecast_month="2025-08")
    print(f"[INFO] pred_df rows: {len(pred_df)}")

    print("=== [6] Create submission (value >= 1000 only) ===")
    submission = create_submission(
        pairs_df=pairs_df,
        pred_df=pred_df,
        models=models,
        feature_cols=feature_cols,
        min_value=1000,
    )

    out_path = OUTPUT_DIR / "submission_v13_lgbm_hybrid_valge1000.csv"
    submission.to_csv(out_path, index=False)
    print(f"[SAVE] {out_path}")


if __name__ == "__main__":
    main()


=== [1] Load monthly & pivot ===
=== [2] Load pairs_v10_best (BayesOpt 필터 반영본) ===
[INFO] pairs_v10_best rows: 2234
=== [3] Build training df (hybrid FE) ===
[INFO] training rows: 83841
=== [4] Train LGBM seed ensemble (MAE objective) ===
[Split] Train: (68203, 23), Valid: (11170, 23)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5125
[LightGBM] [Info] Number of data points in the train set: 68203, number of used features: 23
[LightGBM] [Info] Start training from score 12.939315
[Seed 42] Valid MAE (log1p-space) = 1.03837
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5125
[LightGBM] [Info] Number of data points in the train set: 68203, number of used features: 23
[LightGBM] [Info] Start training from 